# Analyse van energie- en waterverbruik

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
import numpy as np
import pandas as pd
import os, logging, importlib

from datetime import datetime

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file, reset_output, save
from bokeh.models import HoverTool, ColumnDataSource, LabelSet

#importlib.reload(logging)
#logging.basicConfig(level=logging.ERROR, format="%(levelname)s:%(message)s")

In [4]:
datadir = os.path.expanduser('~/Private/Git/Meterstanden/data')
outputdir = os.path.expanduser('~/Private/Dropbox/Documents/Meterstanden')
filename = os.path.join(datadir, 'Meterstanden.csv')

Plot width and height can be set for a specific device such that the figure takes as much real estate as possible.

In [5]:
# The values below match the iPhone 7 or iPad Pro display

plot_width  = 1334   # iPhone 7 width
plot_height = 750    # iPhone 7 height

plot_width  = int(2048 / 2)  # iPad Pro 9.7" logical width
plot_height = int(1536 / 2)  # iPad Pro 9.7" logical height

plot_width  = 900  # on-screen in Jupyter notebook
plot_height = 600  # on-screen in Jupyter notebook

# Read the input file

In [6]:
# Read the input csv file, make the Date_Time column the index column and parse the dates

df = pd.read_csv(filename, index_col="Date_Time", parse_dates=True)

print ("Reading input csv file from {}.".format(filename))

df.info()

Reading input csv file from /Users/rik/Private/Git/Meterstanden/data/Meterstanden.csv.
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1020 entries, 2015-10-07 12:00:00 to 2018-08-26 09:21:00
Data columns (total 8 columns):
Gas            934 non-null float64
eDag           927 non-null float64
eNacht         963 non-null float64
SMA_3000       996 non-null float64
SMA_7000       996 non-null float64
SMA            581 non-null float64
Temperatuur    933 non-null float64
Water          927 non-null float64
dtypes: float64(8)
memory usage: 71.7 KB


# Waterverbruik

In [7]:
fig = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Volume (m^3)', \
           plot_height=plot_height, plot_width=plot_width, title='Verbruik Water')

fig.circle(df.index, df.Water)

reset_output()
output_notebook()
show(fig)

reset_output()
output_file(os.path.join(outputdir, "Verbruik_Water.html"))
save(fig)


Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Verbruik_Water.html'

# Gasverbruik

In [8]:
fig = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Volume (m^3)', \
             plot_height=plot_height, plot_width=plot_width, title='Verbruik Gas sinds 2015')

fig.circle(df.index, df.Gas)

reset_output()
output_notebook()
show(fig)

reset_output()
output_file(os.path.join(outputdir, "Verbruik_Gas.html"))
save(fig)

Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Verbruik_Gas.html'

# Gasverbruik per dag

In [9]:
# Bereken het verbruik in gas tussen twee opeenvolgende metingen

g_diff = df['Gas'].diff().dropna()

# Bereken de tijd tussen twee opeenvolgende metingen in dagen (both options are the same)

time_diff_days = g_diff.index.to_series().diff().astype('timedelta64[s]') / 60. / 60. / 24.
time_diff_days = g_diff.index.to_series().diff().dt.total_seconds().fillna(0) / 60. / 60. / 24.

# Bereken het verbruik van gas per dag

g_per_dag = g_diff / time_diff_days

from bokeh.models import LinearAxis, Range1d

p = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Verbruik (m^3/dag)', \
           plot_height=int(plot_height * 1.5), plot_width=plot_width, title='Verbruik Gas per dag', \
           toolbar_location="above")

p.y_range = Range1d(-20,30)
p.line(g_per_dag.index, g_per_dag, line_width=1)
p.circle(g_per_dag.index, g_per_dag, fill_color='white', size=4)

p.extra_y_ranges = {"temp": Range1d(start=-10, end=100)}
p.add_layout(LinearAxis(y_range_name="temp"), 'right')

p.line(df.index, df.Temperatuur, color='green', y_range_name='temp')
p.circle(df.index, df.Temperatuur, color='green', fill_color='white', size=4, y_range_name='temp')

reset_output()
output_notebook()
show(p)

reset_output()
output_file(os.path.join(outputdir, "Verbruik_Gas_per_dag.html"))
save(p)

Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Verbruik_Gas_per_dag.html'

# Elektriciteitsverbruik

In [10]:
e_total = df['eDag'] + df['eNacht']

p = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Verbruik (kWh)', \
           plot_height=plot_height, plot_width=plot_width, title='Verbruik Elektriciteit sinds 2015')
p.line(e_total.index, e_total)
p.circle(e_total.index, e_total, size=3)

reset_output()
output_notebook()
show(p)

reset_output()
output_file(os.path.join(outputdir, "Verbruik_Elektriciteit.html"))
save(p)


Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Verbruik_Elektriciteit.html'

# Elektriciteit per dag

In [11]:
# Bereken het verbruik in elektriciteit tussen twee opeenvolgende metingen
# Elektriciteit voor de dag en nacht teller eerst bijeen tellen

e_total = df.eDag + df.eNacht
e_diff = e_total.diff().dropna()

# Bereken de tijd tussen twee opeenvolgende metingen in dagen

time_diff_days = e_diff.index.to_series().diff().dt.total_seconds().fillna(0) / 60. / 60. / 24.

# Bereken het elektriciteitsverbruik per dag

e_per_dag = e_diff / time_diff_days

# Bereken de totale opbrengst per dag van de zonnepanelen

z_total = df.SMA_3000.shift(1) + df.SMA_7000.shift(1)
z_total = z_total.dropna()

# Het totaal verbruik per dag is de sum van het elektriciteitsverbruik en de opbrengst van de zonnepanelen

ez_per_dag = e_per_dag + z_total

# Prepare and show the plot

p = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Verbruik (kWh/dag)', \
		   plot_height=plot_height, plot_width=plot_width, title='Verbruik Elektriciteit per dag sinds 2015')

p.line(ez_per_dag.index, ez_per_dag, line_width=1)
p.circle(ez_per_dag.index, ez_per_dag, fill_color='white', size=4)

reset_output()
output_notebook()
show(p)

reset_output()
output_file(os.path.join(outputdir, "Verbruik_Elektriciteit_per_dag.html"))
save(p)


Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Verbruik_Elektriciteit_per_dag.html'

# Verhouding opbrengst zonnepanelen

In [12]:
# De SMA 7000 bevat 32 zonnepanelen, de SMA 3000 bevat er 14. De verhouging zou dus ± 2.28 moeten zijn

sma_ratio = df.SMA_7000 / df.SMA_3000

# Prepare and show the plot

p = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='SMA 7000 / SMA 3000', \
           plot_height=plot_height, plot_width=plot_width, title='Verhouding SMA 7000 vs SMA 3000')

p.circle(sma_ratio.index, sma_ratio)

reset_output()
output_notebook()
show(p)

reset_output()
output_file(os.path.join(outputdir, "Ratio_Zonnepanelen.html"))
save(p)

Loading BokehJS ...

'/Users/rik/Private/Dropbox/Documents/Meterstanden/Ratio_Zonnepanelen.html'

# Verschil tussen het berekende z_totaal en df.SMA

In [13]:
df.SMA = df.SMA.fillna(0.0)

sma_diff = z_total - df.SMA

small_values = sma_diff < 0.0001
sma_diff[small_values] = 0.0

df_data = {'DateTime': df.index, 'sma': sma_diff}

df_tmp = pd.DataFrame(df_data)
df_tmp['tt_date'] = [x.strftime("%Y-%m-%d") for x in df_tmp['DateTime']]
df_tmp['tt_sma'] = ["{:.3f}".format(x) for x in df_tmp['sma']]

source = ColumnDataSource(df_tmp.tail(10))

p = figure(x_axis_label='Datum', x_axis_type='datetime', y_axis_label='Calculated - Tabulated SMA', \
           plot_height=plot_height, plot_width=plot_width, title='Controle op berekening totale SMA', \
           tools='resize,pan,wheel_zoom,box_zoom,reset,previewsave,hover',logo=None)

hover = p.select(dict(type=HoverTool))
hover.tooltips = [('date','@tt_date'), ('value','@tt_sma')]
hover.mode = 'mouse'

labels = LabelSet(x='DateTime', y='sma', text='tt_sma', y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)


p.line('DateTime', 'sma', line_width=1, source=source)
p.circle('DateTime', 'sma', fill_color='white', size=4, source=source)

reset_output()
output_notebook()
show(p)

reset_output()
output_file(os.path.join(outputdir, "Controle_SMA_berekening.html"))
save(p)

print ("Totale opbrengst zonnepanelen [kWh]")
print (z_total.tail(10))


/Users/rik/anaconda3/envs/py36/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: ResizeTool is removed in Bokeh 0.12.7, adding it is a no-op. In the future, accessing ResizeTool will be an error
  warn(message)


Loading BokehJS ...

Totale opbrengst zonnepanelen [kWh]
Date_Time
2018-08-17 05:51:00    51.595
2018-08-18 09:00:00    48.807
2018-08-19 09:00:00    42.415
2018-08-20 05:39:00    53.587
2018-08-21 05:42:00    17.510
2018-08-22 05:41:00    46.046
2018-08-23 06:54:00    45.502
2018-08-24 05:50:00    30.002
2018-08-25 07:48:00    32.094
2018-08-26 09:21:00    41.819
dtype: float64


In [14]:
print (z_total.tail(20))

Date_Time
2018-08-07 08:48:00    63.077
2018-08-08 09:49:00    53.460
2018-08-09 09:20:00    38.458
2018-08-10 09:24:00    23.299
2018-08-11 09:58:00    50.808
2018-08-12 08:57:00    50.922
2018-08-13 08:07:00    53.051
2018-08-14 09:36:00    32.166
2018-08-15 09:10:00    22.198
2018-08-16 05:37:00    24.561
2018-08-17 05:51:00    51.595
2018-08-18 09:00:00    48.807
2018-08-19 09:00:00    42.415
2018-08-20 05:39:00    53.587
2018-08-21 05:42:00    17.510
2018-08-22 05:41:00    46.046
2018-08-23 06:54:00    45.502
2018-08-24 05:50:00    30.002
2018-08-25 07:48:00    32.094
2018-08-26 09:21:00    41.819
dtype: float64
